### ARERA (Autorità di Regolazione per Energia Reti e Ambiente) – Offerte Energia (Portale Offerte Luce e Gas)

# !! 
Eseguire questo codice assicurandosi che il computer non entri mai in modalità standby.

In [1]:
import requests
import pandas as pd
import openpyxl
import time
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [2]:
# URL della pagina
url = 'https://www.ilportaleofferte.it/portaleOfferte/it/confronto-tariffe-prezzi-luce-gas.page'

# Making a GET request
r = requests.get(url)

# check status code for response received
# success code - 200
print(r)

# print content of request
print(r.content)

<Response [200]>
b'\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="it">\r\n    <head>\r\n        <meta charset="utf-8" />\r\n        <meta name="viewport" content="width=device-width, initial-scale=1" />\r\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n        <meta name="description" content="Utilizza questa procedura automatica per un confronto neutrale dei prezzi delle offerte di luce a gas. Troverai la tariffa che pi&#249; si adatta ai tuoi bisogni.">\r\n\t\t<meta name="keywords" content="confronto tariffe luce, confronto tariffe gas, confronta tariffe luce, confronto tariffe energia elettrica, confronta tariffe gas, confronto energia elettrica, confronta prezzi energia elettrica, confronto tariffe luce e gas, confronta prezzi gas, confronta prezzi prezzi luce">\r\n        <link rel="icon" type="image/png" href="/portaleOfferte/resources/static/img/favicon.png" />\r\n        <title>\r\n\t\t\tConfronto tariffe luce e gas: confronta i prezzi | Portale Offerte\r\n   

*Per leggere più facilmente codice HTML → aprire pagina sito → tasto destro → INSPECT*

###  **ENERGIA ELETTRICA**

#### PROFILO CLIENTE 01

**Dati Generali**
- **Tipo di Offerta:** Energia elettrica
- **Comune/CAP:** Bolzano (39100)
- **Tipo di Abitazione:** Casa
- **Numero di Persone Conviventi nell'Abitazione:** 1 (singola persona)
- **Tipo di Mercato:** Mercato Libero

**Caratteristiche dell'Offerta**
- **Prezzo:** Fisso
- **Residenza:** Sì (il cliente è residente)
- **Cliente Vulnerabile:** No (il cliente non rientra tra le categorie vulnerabili)
- **Potenza Contrattuale:** 3 kW
- **Fasce Orarie:** Prezzo per fasce orarie
- **Solo Energie Rinnovabili:** No

**Consumo Energetico**
- **Conoscenza del Consumo Annua:** No (il cliente non conosce il proprio consumo annuo)
- **Periodo di Utilizzo della Fornitura:** Tutto l'anno
- **Illuminazione**: Illuminazione LED

**Elettrodomestici e Utilizzi**

Il cliente utilizza diversi elettrodomestici e apparecchi elettronici, tra cui:
- Cottura: Piano cottura elettrico o a induzione
- Frigorifero
- Forno elettrico
- Congelatore
- Lavastoviglie
- Lavatrice
- Televisione
- Personal computer

# !! 
Inserire qui il CAP o il nome del comune/dei comuni per i quali si desidera scaricare le offerte.

ATTENZIONE: Per comuni di piccole dimensioni, è consigliabile inserire direttamente il nome del comune, poiché lo stesso CAP potrebbe essere condiviso da più comuni.

In [3]:
# Lista dei CAP o nomi dei comuni da utilizzare per estrarre le offerte
lista_comuni = ['39100']

In [4]:
# Lista generale per contenere tutti i dati estratti (utile se si intendono scaricare dati per più comuni)
data_list_totale = []  

# Ciclo attraverso ciascun CAP/comune fornito nella lista
for comune in lista_comuni:

    # Inizializza il driver per il browser Chrome
    driver = webdriver.Chrome()
    driver.get(url)  # Accedi all'URL specificato

    # Attende che il radio button "energiaElettrica" sia visibile e lo clicca
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "energiaElettrica"))).click()

    # Trova il campo di input per il CAP e inserisce il valore del CAP/comune corrente
    cap_field = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'capComune')))
    cap_field.clear()  # Pulisce eventuali valori precedenti
    cap_field.send_keys(comune)  # Inserisce il CAP/comune corrente

    # Breve pausa per garantire che l'interfaccia sia pronta
    time.sleep(1)

    # Attende che le opzioni del menu a tendina diventino visibili
    dropdown_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ui-menu-item-wrapper')))         # '.ui-autocomplete-input'

    # Clicca l'opzione
    ActionChains(driver).move_to_element(dropdown_option).click().perform()                 # driver.execute_script("arguments[0].click();", dropdown_option) 

    # Seleziona l'opzione "casa"
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "casa"))).click()

    # Seleziona l'opzione "prezzo fisso"
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "prezzoFisso"))).click()

    # Seleziona l'opzione per "vulnerabile" come NO
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliVulnerabile")))).select_by_value("NO")

    # Seleziona l'opzione per "residente" come SI
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "residenteSI"))).click()

    # Clicca sul pulsante "Avanti"
    driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-medium.gray.link_btn").click()

    # Seleziona il valore "3" (kW) dall'elemento di selezione per "scegliPotenza"
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliPotenza")))).select_by_value("3")

    # Seleziona "fasce" dal menu a tendina per il tipo di tariffa
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "tipoTariffa")))).select_by_value("fasce")

    # Attende che il radio button per "ferNO" sia visibile e lo seleziona
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ferNO"))).click()

    # Clicca sul radio button per "consumoNO", utilizzando JavaScript per garantire il funzionamento anche in caso di problemi di clic
    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "consumoNO"))))

    # Seleziona "ANNO" dall'elemento di selezione per "usoFornitura"
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "usoFornitura")))).select_by_value("ANNO")

    # Seleziona il numero "1" dall'elemento di selezione per "persone"
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "persone")))).select_by_value("1")

    # Seleziona "ILLLD" dall'elemento di selezione per "tipoIlluminazione"
    Select(WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "tipoIlluminazione")))).select_by_value("ILLLD")

    # Lista degli ID delle checkbox da selezionare
    checkbox_ids = ["cotturaEE", "frigorifero", "forno", "congelatore", "lavastoviglie", "lavatrice", "televisione", "personalComputer"]

    # Ciclo attraverso ciascuna checkbox nella lista
    for checkbox_id in checkbox_ids:
        # Attende che la checkbox sia presente e la recupera
        checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, checkbox_id)))
        
        # Scorre fino alla checkbox per garantire che sia visibile
        driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)

        # Se la checkbox non è già selezionata, prova a cliccarla
        if not checkbox.is_selected():
            try:
                checkbox.click()  # Clicca normalmente sulla checkbox
            except:
                # Se ci sono problemi, utilizza JavaScript per cliccare sulla checkbox
                driver.execute_script("arguments[0].click();", checkbox)

    # Clicca sul pulsante "confronta" per inviare le selezioni
    driver.find_element(By.NAME, "confronta").click()

    ## FILTRO CATEGORIA OFFERTA
    # Seleziona la categoria di offerta "LIBERO"
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "categoriaOfferta")))).select_by_value("LIBERO")

    # Clicca nuovamente sul pulsante di confronto per aggiornare i risultati
    driver.execute_script("arguments[0].click();", driver.find_element(By.NAME, "confronta"))
    time.sleep(1)  # Attende che i risultati siano caricati

    # Lista temporanea per i dati del comune corrente
    data_list = []  

    # Trova tutti i box dei risultati, escludendo il primo elemento
    boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]

    # Inizializzazione contatore
    index = 0

    # Loop attraverso ciascun box finché ci sono box disponibili
    while index < len(boxes):

        # Controlla se ci sono popup e chiudili se presenti
        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            # Nessun popup trovato, procedi
            pass
        
        # Clicca sul link di dettaglio del box corrente
        dettaglio = boxes[index].find_element(By.CLASS_NAME, "linkDettaglio")
        driver.execute_script("arguments[0].click();", dettaglio)

        # Estrai i dettagli dell'offerta dalla pagina
        commodity = driver.find_element(By.XPATH, "//div[strong[text()='Commodity']]").text.replace("Commodity: ", "").strip()
        data = driver.find_element(By.XPATH, "//div[strong[text()='Data']]").text.replace("Data: ", "").strip()
        comune = driver.find_element(By.XPATH, "//div[strong[text()='CAP']]").text.replace("CAP: ", "").strip()
        nome_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Nome offerta:']]").text.replace("Nome offerta: ", "").strip()
        cod_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Codice offerta:']]").text.replace("Codice offerta: ", "").strip()
        venditore = driver.find_element(By.XPATH, "//div[contains(@class, 'venditore_dedtail')]//a").text.strip()
        
        # Estrai le date di validità dell'offerta
        validità = driver.find_element(By.XPATH, "//div[strong[text()='Valida dal']]").text.strip()
        inizio = validità.split('Valida dal')[1].split('al')[0].strip()
        fine = validità.rsplit('al', 1)[1].strip()

        # Estrai i vari costi dalla pagina
        pattern = r'[\d\.]+,\d+|\d+\.?\d*' # per estrarre valori numerici

        spesa_annua = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa annua stimata']]//td[@class='valore']/strong").text.strip())[0]
        materia_prima_energia = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Materia prima energia']]//td[@class='valore']/strong").text.strip())[0]
        trasporto_gestione_contatore = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Trasporto e gestione contatore']]//td[@class='valore']/strong").text.strip())[0]
        oneri_sistema = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Oneri di sistema']]//td[@class='valore']/strong").text.strip())[0]
        spesa_netto_imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa netto imposte stimata']]//td[@class='valore']/strong").text.strip())[0]
        imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Imposte']]//td[@class='valore']/strong").text.strip())[0]

        # Prova a ottenere gli sconti soggetti a IVA, se non disponibili imposta a '0,00'
        try:
            sconti_iva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Sconti una-tantum soggetti a IVA']]//td[@class='valore']/strong").text.strip())[0]
        except:
            sconti_iva = '0,00'

        # Prova a ottenere gli sconti non soggetti a IVA, se non disponibili imposta a '0,00'
        try:
            sconti_NOiva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Sconti una-tantum non soggetti a IVA']]//td[@class='valore']/strong").text.strip())[0]
        except:
            sconti_NOiva = '0,00'

        # Estrai l'IVA e altri dettagli dell'offerta
        iva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA']]//td[@class='valore']/strong").text.strip())[0]
        tipo_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Tipo di offerta']]").text.replace("Tipo di offerta: ", "").strip()
        prezzo = driver.find_element(By.XPATH, "//div[strong[text()='Prezzo']]").text.replace("Prezzo: ", "").strip()
        potenzakW = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Livello di potenza']]").text.strip())[0]
        consumo_annuo = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Consumo annuo']]").text.strip())[0]
        tipo_cliente = driver.find_element(By.XPATH, "//div[strong[text()='Tipo cliente']]").text.replace("Tipo cliente: ", "").strip()
        descrizione = driver.find_element(By.CSS_SELECTOR, "div.boxDettaglio.descr_offerta p").text.strip()

        # Aggiunge i dettagli dell'offerta estratti alla lista dei dati temporanei
        data_list.append({
            "commodity": commodity, 
            "data": data,
            "comune": comune,
            "nome_offerta": nome_offerta,
            "cod_offerta": cod_offerta,
            "venditore": venditore,
            "data_inizio": inizio,
            "data_fine": fine,
            "spesa_annua(euro)": spesa_annua,
            "materia_prima_energia(euro)": materia_prima_energia,
            "trasporto_gestione_contatore(euro)": trasporto_gestione_contatore,
            "oneri_sistema(euro)": oneri_sistema,
            "spesa_netto_imposte(euro)": spesa_netto_imposte,
            "imposte(euro)": imposte,
            "sconti_una-tantum_soggettiIVA(euro)": sconti_iva,
            "IVA(euro)": iva,
            "sconti_una-tantum_nonsoggettiIVA(euro)": sconti_NOiva,
            "tipo_offerta": tipo_offerta,
            "prezzo": prezzo,
            "potenza(kW)": potenzakW,
            "consumo_annuo(kWh)": consumo_annuo,
            "tipo_cliente": tipo_cliente,
            "descrizione": descrizione
        })

        # Torna indietro alla pagina precedente per continuare con il prossimo box
        driver.back()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scorre fino in fondo alla pagina

        # Prova a chiudere eventuali popup
        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass  # Se non ci sono popup, continua l'esecuzione
        
        # Ricarica la lista dei box dei risultati
        boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]
        index += 1  # Incrementa l'indice per passare al prossimo box

    # Aggiungi i dati raccolti del comune corrente alla lista totale
    data_list_totale.extend(data_list)  

    # Chiudi il driver una volta completato il scraping per il comune
    driver.quit()

In [5]:
df01_EE = pd.DataFrame(data_list_totale)
df01_EE = df01_EE.sort_values(by=['comune', 'spesa_annua(euro)'], ascending=[True, True])
df01_EE.insert(1, 'profilo_cliente', '01')
df01_EE.insert(20, 'persone_abitazione', 1)
df01_EE

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,sconti_una-tantum_soggettiIVA(euro),IVA(euro),sconti_una-tantum_nonsoggettiIVA(euro),tipo_offerta,prezzo,persone_abitazione,potenza(kW),consumo_annuo(kWh),tipo_cliente,descrizione
91,Energia elettrica,01,18/10/2024,39100 - Bolzano,LUCE DOMESTICO 1024,027274ESFFL10XX0001024LUCEDOMFIX,VERONA SERVICE S.R.L.,01/10/2024,31/10/2024,"1.297,61",...,"0,00","117,97","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,OFFERTA RISERVATA AI CLIENTI DOMESTICO
0,Energia elettrica,01,18/10/2024,39100 - Bolzano,100% Energia Positiva Casa PRODUTTORE,035700ESFFL01XX000000POCMPVA24SP,ENERGIA POSITIVA SOCIETA COOPERATIVA,23/05/2024,31/12/2024,"569,00",...,"0,00","60,82","100,00",Fissa,Fasce,1,3,2.134,Domestico residente,OFFERTA ATTIVABILE ESCLUSIVAMENTE DA SOCI DI E...
1,Energia elettrica,01,18/10/2024,39100 - Bolzano,G&J DOM ListfissoFasce.Res 001RP2,038424ESFFL07XXDOMFASFISGJRES12R,G&J Energia Futura SRL,01/10/2024,31/10/2024,"595,67",...,"0,00","54,16","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,Le presenti condizioni economiche sono valide ...
2,Energia elettrica,01,18/10/2024,39100 - Bolzano,MITICA ENERGIA BIORARIA,000139ESFFL01XXW71974W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"607,25",...,"144,00","55,2","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,Offerta per clienti retail con tipologia di ut...
3,Energia elettrica,01,18/10/2024,39100 - Bolzano,A2A Full Luce,000294ESFFL01XXA2AFullD241024048,A2A ENERGIA SPA,10/10/2024,10/11/2024,"610,28",...,"70,08","55,48","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,A2A Full Luce è l’offerta a Mercato Libero che...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Energia elettrica,01,18/10/2024,39100 - Bolzano,FIX FAMILY SMALL,003450ESFOL01XX00000010065711024,DUFERCO ENERGIA SPA,11/10/2024,31/10/2024,"880,78",...,"0,00","80,08","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,1006571
87,Energia elettrica,01,18/10/2024,39100 - Bolzano,Smart Luce Fix 3Fasce,040505ESFFL10XXPULSEEFIX76062410,PULSEE S.R.L.,11/10/2024,21/10/2024,"907,22",...,"0,00","82,48","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,Offerta a prezzo fisso. L'energia fornita al/i...
88,Energia elettrica,01,18/10/2024,39100 - Bolzano,FIX_Earthcare_EE_DO_Bio,025280ESFFL00XXXXXXXXCES24AX2XDO,ALPHASTAR,13/09/2024,31/12/2024,"924,79",...,"0,00","84,07","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,All'offerta possono accedere tutti i clienti t...
89,Energia elettrica,01,18/10/2024,39100 - Bolzano,FISSO_EE_DO_Bio,025280ESFFL00XXXXXXXXMAF24AX2XDO,ALPHASTAR,13/09/2024,31/12/2024,"924,79",...,"0,00","84,07","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,All'offerta possono accedere tutti i clienti t...


#### PROFILO CLIENTE 02

**Dati Generali**
- **Tipo di Offerta:** Energia elettrica
- **Comune/CAP:** Bolzano (39100)
- **Tipo di Abitazione:** Casa
- **Numero di Persone Conviventi nell'Abitazione:** 2
- **Tipo di Mercato:** Mercato Libero

**Caratteristiche dell'Offerta**
- **Prezzo:** Fisso
- **Residenza:** Sì (il cliente è residente)
- **Cliente Vulnerabile:** No (il cliente non rientra tra le categorie vulnerabili)
- **Potenza Contrattuale:** 4,5 kW
- **Fasce Orarie:** Prezzo per fasce orarie
- **Solo Energie Rinnovabili:** No

**Consumo Energetico**
- **Conoscenza del Consumo Annua:** No (il cliente non conosce il proprio consumo annuo)
- **Periodo di Utilizzo della Fornitura:** Tutto l'anno
- **Illuminazione**: Illuminazione LED

**Elettrodomestici e Utilizzi**

Il cliente utilizza diversi elettrodomestici e apparecchi elettronici, tra cui:
- Cottura: Piano cottura elettrico o a induzione
- Frigorifero
- Forno elettrico
- Congelatore
- Lavastoviglie
- Lavatrice
- Televisione
- Personal computer

In [6]:
data_list_totale = []  

for comune in lista_comuni:

    driver = webdriver.Chrome()
    driver.get(url) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "energiaElettrica"))).click()

    cap_field = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'capComune')))
    cap_field.clear() 
    cap_field.send_keys(comune) 

    time.sleep(1)

    dropdown_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ui-menu-item-wrapper')))         # '.ui-autocomplete-input'

    ActionChains(driver).move_to_element(dropdown_option).click().perform()                 # driver.execute_script("arguments[0].click();", dropdown_option) 
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "casa"))).click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "prezzoFisso"))).click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliVulnerabile")))).select_by_value("NO")

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "residenteSI"))).click()

    driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-medium.gray.link_btn").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliPotenza")))).select_by_value("4.5")       # Che livello di potenza desideri?

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "tipoTariffa")))).select_by_value("fasce")

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ferNO"))).click()

    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "consumoNO"))))

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "usoFornitura")))).select_by_value("ANNO")

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "persone")))).select_by_value("2")               # Quante persone vivono all'interno dell'abitazione?

    Select(WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "tipoIlluminazione")))).select_by_value("ILLLD")

    checkbox_ids = ["cotturaEE", "frigorifero", "forno", "congelatore", "lavastoviglie", "lavatrice", "televisione", "personalComputer"]

    for checkbox_id in checkbox_ids:

        checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, checkbox_id)))
        driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)

        if not checkbox.is_selected():
            try:
                checkbox.click() 
            except:
                driver.execute_script("arguments[0].click();", checkbox)

    driver.find_element(By.NAME, "confronta").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "categoriaOfferta")))).select_by_value("LIBERO")

    driver.execute_script("arguments[0].click();", driver.find_element(By.NAME, "confronta"))
    time.sleep(1)  

    data_list = []  

    boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]

    index = 0

    while index < len(boxes):

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass
        
        dettaglio = boxes[index].find_element(By.CLASS_NAME, "linkDettaglio")
        driver.execute_script("arguments[0].click();", dettaglio)

        commodity = driver.find_element(By.XPATH, "//div[strong[text()='Commodity']]").text.replace("Commodity: ", "").strip()
        data = driver.find_element(By.XPATH, "//div[strong[text()='Data']]").text.replace("Data: ", "").strip()
        comune = driver.find_element(By.XPATH, "//div[strong[text()='CAP']]").text.replace("CAP: ", "").strip()
        nome_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Nome offerta:']]").text.replace("Nome offerta: ", "").strip()
        cod_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Codice offerta:']]").text.replace("Codice offerta: ", "").strip()
        venditore = driver.find_element(By.XPATH, "//div[contains(@class, 'venditore_dedtail')]//a").text.strip()
        
        validità = driver.find_element(By.XPATH, "//div[strong[text()='Valida dal']]").text.strip()
        inizio = validità.split('Valida dal')[1].split('al')[0].strip()
        fine = validità.rsplit('al', 1)[1].strip()

        pattern = r'[\d\.]+,\d+|\d+\.?\d*'

        spesa_annua = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa annua stimata']]//td[@class='valore']/strong").text.strip())[0]
        materia_prima_energia = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Materia prima energia']]//td[@class='valore']/strong").text.strip())[0]
        trasporto_gestione_contatore = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Trasporto e gestione contatore']]//td[@class='valore']/strong").text.strip())[0]
        oneri_sistema = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Oneri di sistema']]//td[@class='valore']/strong").text.strip())[0]
        spesa_netto_imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa netto imposte stimata']]//td[@class='valore']/strong").text.strip())[0]
        imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Imposte']]//td[@class='valore']/strong").text.strip())[0]

        try:
            sconti_iva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Sconti una-tantum soggetti a IVA']]//td[@class='valore']/strong").text.strip())[0]
        except:
            sconti_iva = '0,00'

        try:
            sconti_NOiva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Sconti una-tantum non soggetti a IVA']]//td[@class='valore']/strong").text.strip())[0]
        except:
            sconti_NOiva = '0,00'

        iva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA']]//td[@class='valore']/strong").text.strip())[0]
        tipo_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Tipo di offerta']]").text.replace("Tipo di offerta: ", "").strip()
        prezzo = driver.find_element(By.XPATH, "//div[strong[text()='Prezzo']]").text.replace("Prezzo: ", "").strip()
        potenzakW = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Livello di potenza']]").text.strip())[0]
        consumo_annuo = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Consumo annuo']]").text.strip())[0]
        tipo_cliente = driver.find_element(By.XPATH, "//div[strong[text()='Tipo cliente']]").text.replace("Tipo cliente: ", "").strip()
        descrizione = driver.find_element(By.CSS_SELECTOR, "div.boxDettaglio.descr_offerta p").text.strip()

        data_list.append({
            "commodity": commodity, 
            "data": data,
            "comune": comune,
            "nome_offerta": nome_offerta,
            "cod_offerta": cod_offerta,
            "venditore": venditore,
            "data_inizio": inizio,
            "data_fine": fine,
            "spesa_annua(euro)": spesa_annua,
            "materia_prima_energia(euro)": materia_prima_energia,
            "trasporto_gestione_contatore(euro)": trasporto_gestione_contatore,
            "oneri_sistema(euro)": oneri_sistema,
            "spesa_netto_imposte(euro)": spesa_netto_imposte,
            "imposte(euro)": imposte,
            "sconti_una-tantum_soggettiIVA(euro)": sconti_iva,
            "IVA(euro)": iva,
            "sconti_una-tantum_nonsoggettiIVA(euro)": sconti_NOiva,
            "tipo_offerta": tipo_offerta,
            "prezzo": prezzo,
            "potenza(kW)": potenzakW,
            "consumo_annuo(kWh)": consumo_annuo,
            "tipo_cliente": tipo_cliente,
            "descrizione": descrizione
        })

        driver.back()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass  

        boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]
        index += 1  

    data_list_totale.extend(data_list)  

    driver.quit()

In [ ]:
df02_EE = pd.DataFrame(data_list_totale)
df02_EE = df02_EE.sort_values(by=['comune', 'spesa_annua(euro)'], ascending=[True, True])
df02_EE.insert(1, 'profilo_cliente', '02')
df02_EE.insert(20, 'persone_abitazione', 2)
df02_EE

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,sconti_una-tantum_soggettiIVA(euro),IVA(euro),sconti_una-tantum_nonsoggettiIVA(euro),tipo_offerta,prezzo,persone_abitazione,potenza(kW),consumo_annuo(kWh),tipo_cliente,descrizione
85,Energia elettrica,02,18/10/2024,39100 - Bolzano,FIX FAMILY SMALL,003450ESFOL01XX00000010065711024,DUFERCO ENERGIA SPA,11/10/2024,31/10/2024,"1.002,14",...,"0,00","91,1","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,1006571
86,Energia elettrica,02,18/10/2024,39100 - Bolzano,Value Luce Fix 3Fasce,040505ESFFL10XXPULSEEFIX76142410,PULSEE S.R.L.,11/10/2024,21/10/2024,"1.021,17",...,"0,00","92,84","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,Offerta a prezzo fisso. L'energia fornita al/i...
87,Energia elettrica,02,18/10/2024,39100 - Bolzano,FIX_Earthcare_EE_DO_Bio,025280ESFFL00XXXXXXXXCES24AX2XDO,ALPHASTAR,13/09/2024,31/12/2024,"1.046,68",...,"0,00","95,15","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,All'offerta possono accedere tutti i clienti t...
88,Energia elettrica,02,18/10/2024,39100 - Bolzano,FISSO_EE_DO_Bio,025280ESFFL00XXXXXXXXMAF24AX2XDO,ALPHASTAR,13/09/2024,31/12/2024,"1.046,68",...,"0,00","95,15","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,All'offerta possono accedere tutti i clienti t...
89,Energia elettrica,02,18/10/2024,39100 - Bolzano,Smart Luce Fix 3Fasce,040505ESFFL10XXPULSEEFIX76062410,PULSEE S.R.L.,11/10/2024,21/10/2024,"1.052,25",...,"0,00","95,66","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,Offerta a prezzo fisso. L'energia fornita al/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Energia elettrica,02,18/10/2024,39100 - Bolzano,Alperia Smile Night & Weekend,000368ESFFL10XXXXXX006X241011XVE,ALPERIA SMART SERVICES SRL,11/10/2024,10/11/2024,"977,29",...,"0,00","88,84","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,L'offerta è rivolta ai Clienti di forniture do...
81,Energia elettrica,02,18/10/2024,39100 - Bolzano,Zero pensieri casa fix,029456ESFFL01XX23051ZeroXpensANT,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"980,53",...,"0,00","89,13","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,Offerta fissa di energia elettrica dedicata a ...
82,Energia elettrica,02,18/10/2024,39100 - Bolzano,FIX FAMILY BLUES,003450ESFOL01XX00000010065811024,DUFERCO ENERGIA SPA,11/10/2024,31/10/2024,"985,24",...,"0,00","89,56","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,1006581
83,Energia elettrica,02,18/10/2024,39100 - Bolzano,FIX FAMILY ROCk,003450ESFOL01XX00000010065791024,DUFERCO ENERGIA SPA,11/10/2024,31/10/2024,"985,24",...,"0,00","89,56","0,00",Fissa,Fasce,2,4.5,2.274,Domestico residente,1006579


#### PROFILO CLIENTE 03

**Dati Generali**
- **Tipo di Offerta:** Energia elettrica
- **Comune/CAP:** Bolzano (39100)
- **Tipo di Abitazione:** Casa
- **Numero di Persone Conviventi nell'Abitazione:** 4
- **Tipo di Mercato:** Mercato Libero

**Caratteristiche dell'Offerta**
- **Prezzo:** Fisso
- **Residenza:** Sì (il cliente è residente)
- **Cliente Vulnerabile:** No (il cliente non rientra tra le categorie vulnerabili)
- **Potenza Contrattuale:** 6 kW
- **Fasce Orarie:** Prezzo per fasce orarie
- **Solo Energie Rinnovabili:** No

**Consumo Energetico**
- **Conoscenza del Consumo Annua:** No (il cliente non conosce il proprio consumo annuo)
- **Periodo di Utilizzo della Fornitura:** Tutto l'anno
- **Illuminazione**: Illuminazione LED

**Elettrodomestici e Utilizzi**

Il cliente utilizza diversi elettrodomestici e apparecchi elettronici, tra cui:
- Cottura: Piano cottura elettrico o a induzione
- Frigorifero
- Forno elettrico
- Congelatore
- Lavastoviglie
- Lavatrice
- Televisione
- Personal computer

In [8]:
data_list_totale = []  

for comune in lista_comuni:

    driver = webdriver.Chrome()
    driver.get(url) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "energiaElettrica"))).click()

    cap_field = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'capComune')))
    cap_field.clear() 
    cap_field.send_keys(comune) 

    time.sleep(1)

    dropdown_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ui-menu-item-wrapper')))         # '.ui-autocomplete-input'

    ActionChains(driver).move_to_element(dropdown_option).click().perform()                 # driver.execute_script("arguments[0].click();", dropdown_option) 
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "casa"))).click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "prezzoFisso"))).click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliVulnerabile")))).select_by_value("NO")

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "residenteSI"))).click()

    driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-medium.gray.link_btn").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliPotenza")))).select_by_value("6")       # Che livello di potenza desideri?

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "tipoTariffa")))).select_by_value("fasce")

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ferNO"))).click()

    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "consumoNO"))))

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "usoFornitura")))).select_by_value("ANNO")

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "persone")))).select_by_value("4")               # Quante persone vivono all'interno dell'abitazione?

    Select(WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "tipoIlluminazione")))).select_by_value("ILLLD")

    checkbox_ids = ["cotturaEE", "frigorifero", "forno", "congelatore", "lavastoviglie", "lavatrice", "televisione", "personalComputer"]

    for checkbox_id in checkbox_ids:

        checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, checkbox_id)))
        driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)

        if not checkbox.is_selected():
            try:
                checkbox.click() 
            except:
                driver.execute_script("arguments[0].click();", checkbox)

    driver.find_element(By.NAME, "confronta").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "categoriaOfferta")))).select_by_value("LIBERO")

    driver.execute_script("arguments[0].click();", driver.find_element(By.NAME, "confronta"))
    time.sleep(1)  

    data_list = []  

    boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]

    index = 0

    while index < len(boxes):

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass
        
        dettaglio = boxes[index].find_element(By.CLASS_NAME, "linkDettaglio")
        driver.execute_script("arguments[0].click();", dettaglio)

        commodity = driver.find_element(By.XPATH, "//div[strong[text()='Commodity']]").text.replace("Commodity: ", "").strip()
        data = driver.find_element(By.XPATH, "//div[strong[text()='Data']]").text.replace("Data: ", "").strip()
        comune = driver.find_element(By.XPATH, "//div[strong[text()='CAP']]").text.replace("CAP: ", "").strip()
        nome_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Nome offerta:']]").text.replace("Nome offerta: ", "").strip()
        cod_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Codice offerta:']]").text.replace("Codice offerta: ", "").strip()
        venditore = driver.find_element(By.XPATH, "//div[contains(@class, 'venditore_dedtail')]//a").text.strip()
        
        validità = driver.find_element(By.XPATH, "//div[strong[text()='Valida dal']]").text.strip()
        inizio = validità.split('Valida dal')[1].split('al')[0].strip()
        fine = validità.rsplit('al', 1)[1].strip()

        pattern = r'[\d\.]+,\d+|\d+\.?\d*'

        spesa_annua = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa annua stimata']]//td[@class='valore']/strong").text.strip())[0]
        materia_prima_energia = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Materia prima energia']]//td[@class='valore']/strong").text.strip())[0]
        trasporto_gestione_contatore = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Trasporto e gestione contatore']]//td[@class='valore']/strong").text.strip())[0]
        oneri_sistema = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Oneri di sistema']]//td[@class='valore']/strong").text.strip())[0]
        spesa_netto_imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa netto imposte stimata']]//td[@class='valore']/strong").text.strip())[0]
        imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Imposte']]//td[@class='valore']/strong").text.strip())[0]

        try:
            sconti_iva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Sconti una-tantum soggetti a IVA']]//td[@class='valore']/strong").text.strip())[0]
        except:
            sconti_iva = '0,00'

        try:
            sconti_NOiva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Sconti una-tantum non soggetti a IVA']]//td[@class='valore']/strong").text.strip())[0]
        except:
            sconti_NOiva = '0,00'

        iva = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA']]//td[@class='valore']/strong").text.strip())[0]
        tipo_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Tipo di offerta']]").text.replace("Tipo di offerta: ", "").strip()
        prezzo = driver.find_element(By.XPATH, "//div[strong[text()='Prezzo']]").text.replace("Prezzo: ", "").strip()
        potenzakW = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Livello di potenza']]").text.strip())[0]
        consumo_annuo = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Consumo annuo']]").text.strip())[0]
        tipo_cliente = driver.find_element(By.XPATH, "//div[strong[text()='Tipo cliente']]").text.replace("Tipo cliente: ", "").strip()
        descrizione = driver.find_element(By.CSS_SELECTOR, "div.boxDettaglio.descr_offerta p").text.strip()

        data_list.append({
            "commodity": commodity, 
            "data": data,
            "comune": comune,
            "nome_offerta": nome_offerta,
            "cod_offerta": cod_offerta,
            "venditore": venditore,
            "data_inizio": inizio,
            "data_fine": fine,
            "spesa_annua(euro)": spesa_annua,
            "materia_prima_energia(euro)": materia_prima_energia,
            "trasporto_gestione_contatore(euro)": trasporto_gestione_contatore,
            "oneri_sistema(euro)": oneri_sistema,
            "spesa_netto_imposte(euro)": spesa_netto_imposte,
            "imposte(euro)": imposte,
            "sconti_una-tantum_soggettiIVA(euro)": sconti_iva,
            "IVA(euro)": iva,
            "sconti_una-tantum_nonsoggettiIVA(euro)": sconti_NOiva,
            "tipo_offerta": tipo_offerta,
            "prezzo": prezzo,
            "potenza(kW)": potenzakW,
            "consumo_annuo(kWh)": consumo_annuo,
            "tipo_cliente": tipo_cliente,
            "descrizione": descrizione
        })

        driver.back()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass  

        boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]
        index += 1  

    data_list_totale.extend(data_list)  

    driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF69BE7B095+29557]
	(No symbol) [0x00007FF69BDEFA50]
	(No symbol) [0x00007FF69BCAB56A]
	(No symbol) [0x00007FF69BCFF695]
	(No symbol) [0x00007FF69BCFF8EC]
	(No symbol) [0x00007FF69BD4B777]
	(No symbol) [0x00007FF69BD271CF]
	(No symbol) [0x00007FF69BD4851C]
	(No symbol) [0x00007FF69BD26F33]
	(No symbol) [0x00007FF69BCF116F]
	(No symbol) [0x00007FF69BCF22D1]
	GetHandleVerifier [0x00007FF69C1AC96D+3378253]
	GetHandleVerifier [0x00007FF69C1F8497+3688311]
	GetHandleVerifier [0x00007FF69C1ED1CB+3642539]
	GetHandleVerifier [0x00007FF69BF3A6B6+813462]
	(No symbol) [0x00007FF69BDFAB5F]
	(No symbol) [0x00007FF69BDF6B74]
	(No symbol) [0x00007FF69BDF6D10]
	(No symbol) [0x00007FF69BDE5C1F]
	BaseThreadInitThunk [0x00007FFD60E7257D+29]
	RtlUserThreadStart [0x00007FFD62AAAF08+40]


In [10]:
df03_EE = pd.DataFrame(data_list_totale)
df03_EE = df03_EE.sort_values(by=['comune', 'spesa_annua(euro)'], ascending=[True, True])
df03_EE.insert(1, 'profilo_cliente', '03')
df03_EE.insert(20, 'persone_abitazione', 4)
df03_EE

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,sconti_una-tantum_soggettiIVA(euro),IVA(euro),sconti_una-tantum_nonsoggettiIVA(euro),tipo_offerta,prezzo,persone_abitazione,potenza(kW),consumo_annuo(kWh),tipo_cliente,descrizione
52,Energia elettrica,03,18/10/2024,39100 - Bolzano,FIX FAMILY POP,003450ESFOL01XX00000010065731024,DUFERCO ENERGIA SPA,11/10/2024,31/10/2024,"1.000,82",...,"0,00","90,98","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,1006573
53,Energia elettrica,03,18/10/2024,39100 - Bolzano,BIENNALE LUCE CASA PROMO,000753ESFFL09XXP0203PB2378240924,SINERGAS S.P.A.,24/09/2024,21/10/2024,"1.001,25",...,"0,00","93,74","30,00",Fissa,Fasce,4,6,2.525,Domestico residente,L'offerta multioraria a prezzo fisso è riserva...
54,Energia elettrica,03,18/10/2024,39100 - Bolzano,PiuControllo Special FLAT Casa Luce SL,000415ESFFL01XX000FLT2GECSLWXV21,Hera Comm S.p.a.,01/10/2024,23/10/2024,"1.001,88",...,"0,00","91,07","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,Passa all'energia che ti assicura un prezzo de...
55,Energia elettrica,03,18/10/2024,39100 - Bolzano,FIX FAMILY SEMPRE ZERO M,003450ESFOL01XX00000010065851024,DUFERCO ENERGIA SPA,11/10/2024,31/10/2024,"1.003,57",...,"0,00","91,22","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,1006585
56,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE CASA SICURO 3 FASCE,025767ESFFL01XXXXXXXEEDOMFN1024T,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.011,04",...,"0,00","91,9","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,Offerta a prezzo fisso.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,Energia elettrica,03,18/10/2024,39100 - Bolzano,Relax Light Casa,000488ESFFL01XX0000000EFIC01024L,BLUENERGY GROUP S.P.A.,13/10/2024,12/11/2024,"990,32",...,"0,00","90,02","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,L’offerta che ti permette di bloccare la compo...
48,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE SICURO,000398ESFFL07XXBDFISILU241006500,Broni Stradella Gas e Luce S.r.l.,04/10/2024,05/11/2024,"991,34",...,"0,00","90,11","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,L'offerta che ti garantisce la stabilità di un...
49,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE SICURO,001293ESFFL07XXEDFISILU241001400,UNOENERGY SPA,04/10/2024,05/11/2024,"991,34",...,"0,00","90,11","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,L'offerta che ti garantisce la stabilità di un...
50,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE SICURO Rata Costante,001293ESFML07XXEDSICORA241002000,UNOENERGY SPA,04/10/2024,05/11/2024,"991,49",...,"0,00","90,13","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,Rata Costante Luce è l’offerta luce di Unoener...


In [11]:
df_EE = pd.concat([df01_EE, df02_EE, df03_EE], ignore_index=True)
df_EE

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,sconti_una-tantum_soggettiIVA(euro),IVA(euro),sconti_una-tantum_nonsoggettiIVA(euro),tipo_offerta,prezzo,persone_abitazione,potenza(kW),consumo_annuo(kWh),tipo_cliente,descrizione
0,Energia elettrica,01,18/10/2024,39100 - Bolzano,LUCE DOMESTICO 1024,027274ESFFL10XX0001024LUCEDOMFIX,VERONA SERVICE S.R.L.,01/10/2024,31/10/2024,"1.297,61",...,"0,00","117,97","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,OFFERTA RISERVATA AI CLIENTI DOMESTICO
1,Energia elettrica,01,18/10/2024,39100 - Bolzano,100% Energia Positiva Casa PRODUTTORE,035700ESFFL01XX000000POCMPVA24SP,ENERGIA POSITIVA SOCIETA COOPERATIVA,23/05/2024,31/12/2024,"569,00",...,"0,00","60,82","100,00",Fissa,Fasce,1,3,2.134,Domestico residente,OFFERTA ATTIVABILE ESCLUSIVAMENTE DA SOCI DI E...
2,Energia elettrica,01,18/10/2024,39100 - Bolzano,G&J DOM ListfissoFasce.Res 001RP2,038424ESFFL07XXDOMFASFISGJRES12R,G&J Energia Futura SRL,01/10/2024,31/10/2024,"595,67",...,"0,00","54,16","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,Le presenti condizioni economiche sono valide ...
3,Energia elettrica,01,18/10/2024,39100 - Bolzano,MITICA ENERGIA BIORARIA,000139ESFFL01XXW71974W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"607,25",...,"144,00","55,2","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,Offerta per clienti retail con tipologia di ut...
4,Energia elettrica,01,18/10/2024,39100 - Bolzano,A2A Full Luce,000294ESFFL01XXA2AFullD241024048,A2A ENERGIA SPA,10/10/2024,10/11/2024,"610,28",...,"70,08","55,48","0,00",Fissa,Fasce,1,3,2.134,Domestico residente,A2A Full Luce è l’offerta a Mercato Libero che...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Energia elettrica,03,18/10/2024,39100 - Bolzano,Relax Light Casa,000488ESFFL01XX0000000EFIC01024L,BLUENERGY GROUP S.P.A.,13/10/2024,12/11/2024,"990,32",...,"0,00","90,02","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,L’offerta che ti permette di bloccare la compo...
272,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE SICURO,000398ESFFL07XXBDFISILU241006500,Broni Stradella Gas e Luce S.r.l.,04/10/2024,05/11/2024,"991,34",...,"0,00","90,11","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,L'offerta che ti garantisce la stabilità di un...
273,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE SICURO,001293ESFFL07XXEDFISILU241001400,UNOENERGY SPA,04/10/2024,05/11/2024,"991,34",...,"0,00","90,11","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,L'offerta che ti garantisce la stabilità di un...
274,Energia elettrica,03,18/10/2024,39100 - Bolzano,LUCE SICURO Rata Costante,001293ESFML07XXEDSICORA241002000,UNOENERGY SPA,04/10/2024,05/11/2024,"991,49",...,"0,00","90,13","0,00",Fissa,Fasce,4,6,2.525,Domestico residente,Rata Costante Luce è l’offerta luce di Unoener...


In [12]:
# Gestione dei separatori di migliaia e decimali e conversione in valori numerici
df_EE['potenza(kW)'] = pd.to_numeric(df_EE['potenza(kW)'])
df_EE['consumo_annuo(kWh)'] = pd.to_numeric(df_EE['consumo_annuo(kWh)'].str.replace('.', ''))
 
for col in df_EE.columns:
    if 'euro' in col:
        df_EE[col] = df_EE[col].str.replace('.', '')
        df_EE[col] = pd.to_numeric(df_EE[col].str.replace('.', '').str.replace(',', '.'))
 
# Gestione colonne data
df_EE['data'] = pd.to_datetime(df_EE['data'], format='%d/%m/%Y').dt.date
df_EE['data_inizio'] = pd.to_datetime(df_EE['data_inizio'], format='%d/%m/%Y').dt.date
df_EE['data_fine'] = pd.to_datetime(df_EE['data_fine'], format='%d/%m/%Y').dt.date

df_EE = df_EE.sort_values(by=['comune', 'profilo_cliente', 'spesa_annua(euro)'], ascending=[True, True, True])
 
df_EE.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276 entries, 1 to 223
Data columns (total 25 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   commodity                               276 non-null    object 
 1   profilo_cliente                         276 non-null    object 
 2   data                                    276 non-null    object 
 3   comune                                  276 non-null    object 
 4   nome_offerta                            276 non-null    object 
 5   cod_offerta                             276 non-null    object 
 6   venditore                               276 non-null    object 
 7   data_inizio                             276 non-null    object 
 8   data_fine                               276 non-null    object 
 9   spesa_annua(euro)                       276 non-null    float64
 10  materia_prima_energia(euro)             276 non-null    float64
 11

In [13]:
df_EE

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,sconti_una-tantum_soggettiIVA(euro),IVA(euro),sconti_una-tantum_nonsoggettiIVA(euro),tipo_offerta,prezzo,persone_abitazione,potenza(kW),consumo_annuo(kWh),tipo_cliente,descrizione
1,Energia elettrica,01,2024-10-18,39100 - Bolzano,100% Energia Positiva Casa PRODUTTORE,035700ESFFL01XX000000POCMPVA24SP,ENERGIA POSITIVA SOCIETA COOPERATIVA,2024-05-23,2024-12-31,569.00,...,0.00,60.82,100.0,Fissa,Fasce,1,3.0,2134,Domestico residente,OFFERTA ATTIVABILE ESCLUSIVAMENTE DA SOCI DI E...
2,Energia elettrica,01,2024-10-18,39100 - Bolzano,G&J DOM ListfissoFasce.Res 001RP2,038424ESFFL07XXDOMFASFISGJRES12R,G&J Energia Futura SRL,2024-10-01,2024-10-31,595.67,...,0.00,54.16,0.0,Fissa,Fasce,1,3.0,2134,Domestico residente,Le presenti condizioni economiche sono valide ...
3,Energia elettrica,01,2024-10-18,39100 - Bolzano,MITICA ENERGIA BIORARIA,000139ESFFL01XXW71974W4124W00000,Dolomiti Energia SpA,2024-10-07,2024-11-07,607.25,...,144.00,55.20,0.0,Fissa,Fasce,1,3.0,2134,Domestico residente,Offerta per clienti retail con tipologia di ut...
4,Energia elettrica,01,2024-10-18,39100 - Bolzano,A2A Full Luce,000294ESFFL01XXA2AFullD241024048,A2A ENERGIA SPA,2024-10-10,2024-11-10,610.28,...,70.08,55.48,0.0,Fissa,Fasce,1,3.0,2134,Domestico residente,A2A Full Luce è l’offerta a Mercato Libero che...
5,Energia elettrica,01,2024-10-18,39100 - Bolzano,TeaWeb LUCE FIX,001227ESFFL07XX00000002024T05052,Tea Energia s.r.l.,2024-10-15,2024-11-13,633.06,...,0.00,57.56,0.0,Fissa,Fasce,1,3.0,2134,Domestico residente,TeaWeb LUCE FIX ha tutto quello di cui hai bis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,Energia elettrica,03,2024-10-18,39100 - Bolzano,FIX_Earthcare_EE_DO_Bio,025280ESFFL00XXXXXXXXCES24AX2XDO,ALPHASTAR,2024-09-13,2024-12-31,1155.21,...,0.00,105.01,0.0,Fissa,Fasce,4,6.0,2525,Domestico residente,All'offerta possono accedere tutti i clienti t...
220,Energia elettrica,03,2024-10-18,39100 - Bolzano,FISSO_EE_DO_Bio,025280ESFFL00XXXXXXXXMAF24AX2XDO,ALPHASTAR,2024-09-13,2024-12-31,1155.21,...,0.00,105.01,0.0,Fissa,Fasce,4,6.0,2525,Domestico residente,All'offerta possono accedere tutti i clienti t...
221,Energia elettrica,03,2024-10-18,39100 - Bolzano,Smart Luce Fix 3Fasce,040505ESFFL10XXPULSEEFIX76062410,PULSEE S.R.L.,2024-10-11,2024-10-21,1182.80,...,0.00,107.52,0.0,Fissa,Fasce,4,6.0,2525,Domestico residente,Offerta a prezzo fisso. L'energia fornita al/i...
222,Energia elettrica,03,2024-10-18,39100 - Bolzano,Luce Amica Domestico – FIX,020443ESFFL01XXLUCEAMICADO102024,NWG ENERGIA S.P.A. SOCIETÀ BENEFIT,2024-10-01,2024-10-31,1197.45,...,0.00,108.86,0.0,Fissa,Fasce,4,6.0,2525,Domestico residente,L'Offerta Luce Amica domestico – FIX prevede u...


In [14]:
if df_EE['comune'].nunique() == 1:
    df_EE.to_excel(f'ARERA_{df_EE["commodity"].iloc[0].title().replace(" ", "")}-{df_EE["comune"].iloc[0].title().replace(" ", "")}-{df_EE["data"].iloc[0].strftime("%Y-%m-%d")}.xlsx', index=False)
else:
    df_EE.to_excel(f'ARERA_{df_EE["commodity"].iloc[0].title().replace(" ", "")}-{df_EE["data"].iloc[0].strftime("%Y-%m-%d")}.xlsx', index=False)

In [15]:
'''
# check offerte che differiscono tra i vari comuni 
print(df_EE.groupby('comune').size())
df_EE[df_EE['cod_offerta'].isin(df_EE.drop(columns=['comune']).drop_duplicates(keep=False)['cod_offerta'])]

'''

"\n# check offerte che differiscono tra i vari comuni \nprint(df_EE.groupby('comune').size())\ndf_EE[df_EE['cod_offerta'].isin(df_EE.drop(columns=['comune']).drop_duplicates(keep=False)['cod_offerta'])]\n\n"

###  **GAS**

#### PROFILO CLIENTE 01

**Dati Generali**
- **Tipo di Offerta:** Gas
- **Comune/CAP:** Bolzano (39100)
- **Tipo di Abitazione:** Casa
- **Numero di Persone Conviventi nell'Abitazione:** 1
- **Tipo di Mercato:** Mercato Libero

**Caratteristiche dell'Offerta**
- **Prezzo:** Fisso
- **Cliente Vulnerabile:** No

**Consumo e Utilizzo del Gas**
- **Conoscenza del Consumo Annua:** No (il cliente non conosce il proprio consumo annuo)
- **Periodo di Utilizzo della Fornitura:** Tutto l'anno
- **Destinazione d'Uso**: Cottura cibi, produzione di acqua calda sanitaria, riscaldamento
- **Classe del Contatore**: Fino a G6
- **Superficie dell'Abitazione**: Da 40 a 55 m²

In [16]:
data_list_totale = []  

for comune in lista_comuni:

    driver = webdriver.Chrome()
    driver.get(url) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gas"))).click()

    cap_field = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'capComune')))
    cap_field.clear() 
    cap_field.send_keys(comune) 

    time.sleep(1)

    dropdown_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ui-menu-item-wrapper')))         # '.ui-autocomplete-input'

    ActionChains(driver).move_to_element(dropdown_option).click().perform()                 # driver.execute_script("arguments[0].click();", dropdown_option) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "casa"))).click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "prezzoFisso"))).click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliVulnerabile")))).select_by_value("NO")

    driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-medium.gray.link_btn").click()

    checkbox_ids = ["cottura", "acquaCalda", "riscaldamento"] 

    for checkbox_id in checkbox_ids:

        checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, checkbox_id)))
        driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)

        if not checkbox.is_selected():
            try:
                checkbox.click() 
            except:
                driver.execute_script("arguments[0].click();", checkbox)

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliClasse")))).select_by_value("1")   ## Seleziona la classe del contatore (1 = Fino a G6) 
    
    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "consumoGasNO"))))  

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "mesiGas")))).select_by_value("ANNO")   ## Per quanti mesi all'anno verrà utilizzata la fornitura?
    
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "personeGas")))).select_by_value("1")   ## Quante persone vivono all'interno dell'abitazione?
    
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "superficie")))).select_by_value("0")   ## Quanto misura la superficie dell'abitazione? (0 = Da 40 a 55)
    
    driver.find_element(By.NAME, "confronta").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "categoriaOfferta")))).select_by_value("LIBERO")   ## Filtro: Mercato Libero

    driver.execute_script("arguments[0].click();", driver.find_element(By.NAME, "confronta"))
    time.sleep(1)  

    data_list = []  

    boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]

    index = 0

    while index < len(boxes):

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass
        
        dettaglio = boxes[index].find_element(By.CLASS_NAME, "linkDettaglio")
        driver.execute_script("arguments[0].click();", dettaglio)

        commodity = driver.find_element(By.XPATH, "//div[strong[text()='Commodity']]").text.replace("Commodity: ", "").strip()
        data = driver.find_element(By.XPATH, "//div[strong[text()='Data']]").text.replace("Data: ", "").strip()
        comune = cap_nome_comune        # driver.find_element(By.XPATH, "//div[strong[text()='CAP']]").text.replace("CAP: ", "").strip()
        nome_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Nome offerta:']]").text.replace("Nome offerta: ", "").strip()
        cod_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Codice offerta:']]").text.replace("Codice offerta: ", "").strip()
        venditore = driver.find_element(By.XPATH, "//div[contains(@class, 'venditore_dedtail')]//a").text.strip()
        
        validità = driver.find_element(By.XPATH, "//div[strong[text()='Valida dal']]").text.strip()
        inizio = validità.split('Valida dal')[1].split('al')[0].strip()
        fine = validità.rsplit('al', 1)[1].strip()
        
        pattern = r'[\d\.]+,\d+|\d+\.?\d*'

        spesa_annua = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa annua stimata']]//td[@class='valore']/strong").text.strip())[0]
        materia_gas_naturale = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Materia gas naturale']]//td[@class='valore']/strong").text.strip())[0]
        trasporto_gestione_contatore = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Trasporto e gestione contatore']]//td[@class='valore']/strong").text.strip())[0]
        oneri_sistema = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Oneri di sistema']]//td[@class='valore']/strong").text.strip())[0]
        spesa_netto_imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa netto imposte stimata']]//td[@class='valore']/strong").text.strip())[0]
        imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Imposte']]//td[@class='valore']/strong").text.strip())[0]

        iva_5 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 5%']]//td[@class='valore']/strong").text.strip())[0]
        iva_10 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 10%']]//td[@class='valore']/strong").text.strip())[0]
        iva_22 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 22%']]//td[@class='valore']/strong").text.strip())[0]

        tipo_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Tipo di offerta']]").text.replace("Tipo di offerta: ", "").strip()
        classe_contatore = driver.find_element(By.XPATH, "//div[strong[text()='Classe contatore']]").text.replace("Classe contatore: ", "").strip()
        consumo_annuo = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Consumo annuo']]").text.strip())[0]
        tipo_cliente = driver.find_element(By.XPATH, "//div[strong[text()='Tipo cliente']]").text.replace("Tipo cliente: ", "").strip()
        
        descrizione = driver.find_element(By.CSS_SELECTOR, "div.boxDettaglio.descr_offerta p").text.strip()

        data_list.append({
            "commodity": commodity, 
            "data": data,
            "comune": comune,
            "nome_offerta": nome_offerta,
            "cod_offerta": cod_offerta,
            "venditore": venditore,
            "data_inizio": inizio,
            "data_fine": fine,
            "spesa_annua(euro)": spesa_annua,
            "materia_gas_naturale(euro)": materia_gas_naturale,
            "trasporto_gestione_contatore(euro)": trasporto_gestione_contatore,
            "oneri_sistema(euro)": oneri_sistema,
            "spesa_netto_imposte(euro)": spesa_netto_imposte,
            "imposte(euro)": imposte,
            "iva_5(euro)": iva_5,
            "iva_10(euro)": iva_10,
            "iva_22(euro)": iva_22,
            "tipo_offerta": tipo_offerta,
            "classe_contatore": classe_contatore, 
            "consumo_annuo(Smc)": consumo_annuo,
            "tipo_cliente": tipo_cliente,
            "descrizione": descrizione
        })

        driver.back()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass  

        boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]
        index += 1  

    data_list_totale.extend(data_list)  

    driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF69BE7B095+29557]
	(No symbol) [0x00007FF69BDEFA50]
	(No symbol) [0x00007FF69BCAB56A]
	(No symbol) [0x00007FF69BCFF695]
	(No symbol) [0x00007FF69BCFF8EC]
	(No symbol) [0x00007FF69BD4B777]
	(No symbol) [0x00007FF69BD271CF]
	(No symbol) [0x00007FF69BD4851C]
	(No symbol) [0x00007FF69BD26F33]
	(No symbol) [0x00007FF69BCF116F]
	(No symbol) [0x00007FF69BCF22D1]
	GetHandleVerifier [0x00007FF69C1AC96D+3378253]
	GetHandleVerifier [0x00007FF69C1F8497+3688311]
	GetHandleVerifier [0x00007FF69C1ED1CB+3642539]
	GetHandleVerifier [0x00007FF69BF3A6B6+813462]
	(No symbol) [0x00007FF69BDFAB5F]
	(No symbol) [0x00007FF69BDF6B74]
	(No symbol) [0x00007FF69BDF6D10]
	(No symbol) [0x00007FF69BDE5C1F]
	BaseThreadInitThunk [0x00007FFD60E7257D+29]
	RtlUserThreadStart [0x00007FFD62AAAF08+40]


In [16]:
df01_GAS = pd.DataFrame(data_list_totale)
df01_GAS = df01_GAS.sort_values(by=['comune', 'spesa_annua(euro)'], ascending=[True, True])
df01_GAS.insert(1, 'profilo_cliente', '01')
df01_GAS.insert(20, 'persone_abitazione', 1)
df01_GAS.insert(21, 'superficie_abitazione(m2)', 'Da 40 a 55')
df01_GAS

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,iva_5(euro),iva_10(euro),iva_22(euro),tipo_offerta,classe_contatore,persone_abitazione,superficie_abitazione(m2),consumo_annuo(Smc),tipo_cliente,descrizione
0,Gas,01,18/10/2024,39100 - Bolzano,GAS NATURALE FIX,025767GSFML01XXXXXGASDOMFCCC0924,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.301,23",...,"0,00","40,57","154,18",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta a prezzo fisso.
1,Gas,01,18/10/2024,39100 - Bolzano,FAMILY GAS WEB,000139GSFML01XXW71987W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"1.301,26",...,"0,00","39,46","156,37",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta per clienti retail con tipologia di ut...
2,Gas,01,18/10/2024,39100 - Bolzano,E.ON GasClick - Amico new,000362GSFML01XXMP04W0DA00000A000,E.ON Energia S.p.A.,11/10/2024,24/10/2024,"1.306,01",...,"0,00","42,68","150,86",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta di gas naturale per la casa con un pre...
3,Gas,01,18/10/2024,39100 - Bolzano,NeN Gas Special 36,029748GSFML01XX241010GDBSP360000,YADA ENERGIA S.R.L,10/10/2024,23/10/2024,"1.316,94",...,"0,00","43,06","158,55",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Con NeN paghi una rata fissa mensile calcolata...
4,Gas,01,18/10/2024,39100 - Bolzano,GAS FISSA WEB,025767GSVML01XXXXXGASDOMFWEB1024,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.320,89",...,"0,00","41,77","155,35",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta a prezzo fisso.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Gas,01,18/10/2024,39100 - Bolzano,CASA SICURA,029456GSFML01XX23125CASAXSICUEPN,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"1.753,07",...,"0,00","54,82","207,38",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta fissa di gas naturale dedicata a clien...
137,Gas,01,18/10/2024,39100 - Bolzano,CASA SICURA COSTANTE,029456GSFML01XX23126CASAXSICUEPN,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"1.778,56",...,"0,00","55,78","210,07",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta fissa di gas naturale dedicata a clien...
138,Gas,01,18/10/2024,39100 - Bolzano,Zero pensieri casa plus fix,029456GSFML01XX23059ZeroXpensANT,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"1.783,29",...,"0,00","54,58","213,30",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta fissa di gas naturale dedicata a clien...
139,Gas,01,18/10/2024,39100 - Bolzano,CASA PLUS FIX,000453GSFML01XX22799CASAXPLUSSEV,SEGNOVERDE S.P.A.,13/09/2024,31/10/2024,"1.839,01",...,"0,00","55,3","221,92",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta fissa di gas naturale dedicata a clien...


#### PROFILO CLIENTE 02

**Dati Generali**
- **Tipo di Offerta:** Gas
- **Comune/CAP:** Bolzano (39100)
- **Tipo di Abitazione:** Casa
- **Numero di Persone Conviventi nell'Abitazione:** 2
- **Tipo di Mercato:** Mercato Libero

**Caratteristiche dell'Offerta**
- **Prezzo:** Fisso
- **Cliente Vulnerabile:** No

**Consumo e Utilizzo del Gas**
- **Conoscenza del Consumo Annua:** No (il cliente non conosce il proprio consumo annuo)
- **Periodo di Utilizzo della Fornitura:** Tutto l'anno
- **Destinazione d'Uso**: Cottura cibi, produzione di acqua calda sanitaria, riscaldamento
- **Classe del Contatore**: Fino a G6
- **Superficie dell'Abitazione**: Da 60 a 75 m²

In [17]:
data_list_totale = []  

for comune in lista_comuni:

    driver = webdriver.Chrome()
    driver.get(url) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gas"))).click()

    cap_field = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'capComune')))
    cap_field.clear() 
    cap_field.send_keys(comune) 

    time.sleep(1)

    dropdown_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ui-menu-item-wrapper')))         # '.ui-autocomplete-input'

    ActionChains(driver).move_to_element(dropdown_option).click().perform()                 # driver.execute_script("arguments[0].click();", dropdown_option) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "casa"))).click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "prezzoFisso"))).click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliVulnerabile")))).select_by_value("NO")

    driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-medium.gray.link_btn").click()

    checkbox_ids = ["cottura", "acquaCalda", "riscaldamento"] 

    for checkbox_id in checkbox_ids:

        checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, checkbox_id)))
        driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)

        if not checkbox.is_selected():
            try:
                checkbox.click() 
            except:
                driver.execute_script("arguments[0].click();", checkbox)

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliClasse")))).select_by_value("1") 
    
    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "consumoGasNO"))))  

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "mesiGas")))).select_by_value("ANNO") 
    
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "personeGas")))).select_by_value("2")   ## Quante persone vivono all'interno dell'abitazione?
    
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "superficie")))).select_by_value("1")   ## Quanto misura la superficie dell'abitazione?
    
    driver.find_element(By.NAME, "confronta").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "categoriaOfferta")))).select_by_value("LIBERO") 

    driver.execute_script("arguments[0].click();", driver.find_element(By.NAME, "confronta"))
    time.sleep(1)  

    data_list = []  

    boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]

    index = 0

    while index < len(boxes):

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass
        
        dettaglio = boxes[index].find_element(By.CLASS_NAME, "linkDettaglio")
        driver.execute_script("arguments[0].click();", dettaglio)

        commodity = driver.find_element(By.XPATH, "//div[strong[text()='Commodity']]").text.replace("Commodity: ", "").strip()
        data = driver.find_element(By.XPATH, "//div[strong[text()='Data']]").text.replace("Data: ", "").strip()
        comune = cap_nome_comune        # driver.find_element(By.XPATH, "//div[strong[text()='CAP']]").text.replace("CAP: ", "").strip()
        nome_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Nome offerta:']]").text.replace("Nome offerta: ", "").strip()
        cod_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Codice offerta:']]").text.replace("Codice offerta: ", "").strip()
        venditore = driver.find_element(By.XPATH, "//div[contains(@class, 'venditore_dedtail')]//a").text.strip()
        
        validità = driver.find_element(By.XPATH, "//div[strong[text()='Valida dal']]").text.strip()
        inizio = validità.split('Valida dal')[1].split('al')[0].strip()
        fine = validità.rsplit('al', 1)[1].strip()
        
        pattern = r'[\d\.]+,\d+|\d+\.?\d*'

        spesa_annua = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa annua stimata']]//td[@class='valore']/strong").text.strip())[0]
        materia_gas_naturale = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Materia gas naturale']]//td[@class='valore']/strong").text.strip())[0]
        trasporto_gestione_contatore = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Trasporto e gestione contatore']]//td[@class='valore']/strong").text.strip())[0]
        oneri_sistema = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Oneri di sistema']]//td[@class='valore']/strong").text.strip())[0]
        spesa_netto_imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa netto imposte stimata']]//td[@class='valore']/strong").text.strip())[0]
        imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Imposte']]//td[@class='valore']/strong").text.strip())[0]

        iva_5 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 5%']]//td[@class='valore']/strong").text.strip())[0]
        iva_10 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 10%']]//td[@class='valore']/strong").text.strip())[0]
        iva_22 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 22%']]//td[@class='valore']/strong").text.strip())[0]

        tipo_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Tipo di offerta']]").text.replace("Tipo di offerta: ", "").strip()
        classe_contatore = driver.find_element(By.XPATH, "//div[strong[text()='Classe contatore']]").text.replace("Classe contatore: ", "").strip()
        consumo_annuo = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Consumo annuo']]").text.strip())[0]
        tipo_cliente = driver.find_element(By.XPATH, "//div[strong[text()='Tipo cliente']]").text.replace("Tipo cliente: ", "").strip()
        
        descrizione = driver.find_element(By.CSS_SELECTOR, "div.boxDettaglio.descr_offerta p").text.strip()

        data_list.append({
            "commodity": commodity, 
            "data": data,
            "comune": comune,
            "nome_offerta": nome_offerta,
            "cod_offerta": cod_offerta,
            "venditore": venditore,
            "data_inizio": inizio,
            "data_fine": fine,
            "spesa_annua(euro)": spesa_annua,
            "materia_gas_naturale(euro)": materia_gas_naturale,
            "trasporto_gestione_contatore(euro)": trasporto_gestione_contatore,
            "oneri_sistema(euro)": oneri_sistema,
            "spesa_netto_imposte(euro)": spesa_netto_imposte,
            "imposte(euro)": imposte,
            "iva_5(euro)": iva_5,
            "iva_10(euro)": iva_10,
            "iva_22(euro)": iva_22,
            "tipo_offerta": tipo_offerta,
            "classe_contatore": classe_contatore, 
            "consumo_annuo(Smc)": consumo_annuo,
            "tipo_cliente": tipo_cliente,
            "descrizione": descrizione
        })

        driver.back()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass  

        boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]
        index += 1  

    data_list_totale.extend(data_list)  

    driver.quit()

In [18]:
df02_GAS = pd.DataFrame(data_list_totale)
df02_GAS = df02_GAS.sort_values(by=['comune', 'spesa_annua(euro)'], ascending=[True, True])
df02_GAS.insert(1, 'profilo_cliente', '02')
df02_GAS.insert(20, 'persone_abitazione', 2)
df02_GAS.insert(21, 'superficie_abitazione(m2)', 'Da 60 a 75')
df02_GAS

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,iva_5(euro),iva_10(euro),iva_22(euro),tipo_offerta,classe_contatore,persone_abitazione,superficie_abitazione(m2),consumo_annuo(Smc),tipo_cliente,descrizione
0,Gas,02,18/10/2024,39100 - Bolzano,FAMILY GAS WEB,000139GSFML01XXW71987W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"1.684,72",...,"0,00","39,46","225,52",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta per clienti retail con tipologia di ut...
1,Gas,02,18/10/2024,39100 - Bolzano,GAS NATURALE FIX,025767GSFML01XXXXXGASDOMFCCC0924,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.694,99",...,"0,00","40,57","225,19",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta a prezzo fisso.
2,Gas,02,18/10/2024,39100 - Bolzano,E.ON GasClick - Amico new,000362GSFML01XXMP04W0DA00000A000,E.ON Energia S.p.A.,11/10/2024,24/10/2024,"1.719,51",...,"0,00","42,68","225,42",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta di gas naturale per la casa con un pre...
3,Gas,02,18/10/2024,39100 - Bolzano,GAS FISSA WEB,025767GSVML01XXXXXGASDOMFWEB1024,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.725,88",...,"0,00","41,77","228,38",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta a prezzo fisso.
4,Gas,02,18/10/2024,39100 - Bolzano,NeN Gas Special 36,029748GSFML01XX241010GDBSP360000,YADA ENERGIA S.R.L,10/10/2024,23/10/2024,"1.734,07",...,"0,00","43,06","233,77",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Con NeN paghi una rata fissa mensile calcolata...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Gas,02,18/10/2024,39100 - Bolzano,ENEL MOVE GAS,000142GSFML01XXResECAn0000096027,Enel Energia S.p.A.,15/10/2024,29/10/2024,"2.287,98",...,"0,00","56,26","300,98",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,E' l'offerta di Gas naturale dedicata ai siti ...
137,Gas,02,18/10/2024,39100 - Bolzano,Zero pensieri casa plus fix,029456GSFML01XX23059ZeroXpensANT,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"2.308,18",...,"0,00","54,58","307,96",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta fissa di gas naturale dedicata a clien...
138,Gas,02,18/10/2024,39100 - Bolzano,CASA SICURA COSTANTE,029456GSFML01XX23126CASAXSICUEPN,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"2.314,67",...,"0,00","55,78","306,75",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta fissa di gas naturale dedicata a clien...
139,Gas,02,18/10/2024,39100 - Bolzano,CASA PLUS FIX,000453GSFML01XX22799CASAXPLUSSEV,SEGNOVERDE S.P.A.,13/09/2024,31/10/2024,"2.370,63",...,"0,00","55,3","317,79",Fissa,Portata < G6,2,Da 60 a 75,1.460,Domestico,Offerta fissa di gas naturale dedicata a clien...


#### PROFILO CLIENTE 03

**Dati Generali**
- **Tipo di Offerta:** Gas
- **Comune/CAP:** Bolzano (39100)
- **Tipo di Abitazione:** Casa
- **Numero di Persone Conviventi nell'Abitazione:** 4
- **Tipo di Mercato:** Mercato Libero

**Caratteristiche dell'Offerta**
- **Prezzo:** Fisso
- **Cliente Vulnerabile:** No

**Consumo e Utilizzo del Gas**
- **Conoscenza del Consumo Annua:** No (il cliente non conosce il proprio consumo annuo)
- **Periodo di Utilizzo della Fornitura:** Tutto l'anno
- **Destinazione d'Uso**: Cottura cibi, produzione di acqua calda sanitaria, riscaldamento
- **Classe del Contatore**: Fino a G6
- **Superficie dell'Abitazione**: Da 100 a 115 m²

In [20]:
data_list_totale = []  

for comune in lista_comuni:

    driver = webdriver.Chrome()
    driver.get(url) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "gas"))).click()

    cap_field = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'capComune')))
    cap_field.clear() 
    cap_field.send_keys(comune) 

    time.sleep(1)

    dropdown_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ui-menu-item-wrapper')))         # '.ui-autocomplete-input'

    ActionChains(driver).move_to_element(dropdown_option).click().perform()                 # driver.execute_script("arguments[0].click();", dropdown_option) 

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "casa"))).click()

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "prezzoFisso"))).click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliVulnerabile")))).select_by_value("NO")

    driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-medium.gray.link_btn").click()

    checkbox_ids = ["cottura", "acquaCalda", "riscaldamento"] 

    for checkbox_id in checkbox_ids:

        checkbox = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, checkbox_id)))
        driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)

        if not checkbox.is_selected():
            try:
                checkbox.click() 
            except:
                driver.execute_script("arguments[0].click();", checkbox)

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "scegliClasse")))).select_by_value("1") 
    
    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "consumoGasNO"))))  

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "mesiGas")))).select_by_value("ANNO") 
    
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "personeGas")))).select_by_value("4")   ## Quante persone vivono all'interno dell'abitazione?
    
    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "superficie")))).select_by_value("3")   ## Quanto misura la superficie dell'abitazione?
    
    driver.find_element(By.NAME, "confronta").click()

    Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "categoriaOfferta")))).select_by_value("LIBERO") 

    driver.execute_script("arguments[0].click();", driver.find_element(By.NAME, "confronta"))
    time.sleep(1)  

    data_list = []  

    boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]

    index = 0

    while index < len(boxes):

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass
        
        dettaglio = boxes[index].find_element(By.CLASS_NAME, "linkDettaglio")
        driver.execute_script("arguments[0].click();", dettaglio)

        commodity = driver.find_element(By.XPATH, "//div[strong[text()='Commodity']]").text.replace("Commodity: ", "").strip()
        data = driver.find_element(By.XPATH, "//div[strong[text()='Data']]").text.replace("Data: ", "").strip()
        comune = cap_nome_comune        # driver.find_element(By.XPATH, "//div[strong[text()='CAP']]").text.replace("CAP: ", "").strip()
        nome_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Nome offerta:']]").text.replace("Nome offerta: ", "").strip()
        cod_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Codice offerta:']]").text.replace("Codice offerta: ", "").strip()
        venditore = driver.find_element(By.XPATH, "//div[contains(@class, 'venditore_dedtail')]//a").text.strip()
        
        validità = driver.find_element(By.XPATH, "//div[strong[text()='Valida dal']]").text.strip()
        inizio = validità.split('Valida dal')[1].split('al')[0].strip()
        fine = validità.rsplit('al', 1)[1].strip()
        
        pattern = r'[\d\.]+,\d+|\d+\.?\d*'

        spesa_annua = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa annua stimata']]//td[@class='valore']/strong").text.strip())[0]
        materia_gas_naturale = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Materia gas naturale']]//td[@class='valore']/strong").text.strip())[0]
        trasporto_gestione_contatore = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Trasporto e gestione contatore']]//td[@class='valore']/strong").text.strip())[0]
        oneri_sistema = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Oneri di sistema']]//td[@class='valore']/strong").text.strip())[0]
        spesa_netto_imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Spesa netto imposte stimata']]//td[@class='valore']/strong").text.strip())[0]
        imposte = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='Imposte']]//td[@class='valore']/strong").text.strip())[0]

        iva_5 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 5%']]//td[@class='valore']/strong").text.strip())[0]
        iva_10 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 10%']]//td[@class='valore']/strong").text.strip())[0]
        iva_22 = re.findall(pattern, driver.find_element(By.XPATH, "//tr[th[text()='IVA 22%']]//td[@class='valore']/strong").text.strip())[0]

        tipo_offerta = driver.find_element(By.XPATH, "//div[strong[text()='Tipo di offerta']]").text.replace("Tipo di offerta: ", "").strip()
        classe_contatore = driver.find_element(By.XPATH, "//div[strong[text()='Classe contatore']]").text.replace("Classe contatore: ", "").strip()
        consumo_annuo = re.findall(pattern, driver.find_element(By.XPATH, "//div[strong[text()='Consumo annuo']]").text.strip())[0]
        tipo_cliente = driver.find_element(By.XPATH, "//div[strong[text()='Tipo cliente']]").text.replace("Tipo cliente: ", "").strip()
        
        descrizione = driver.find_element(By.CSS_SELECTOR, "div.boxDettaglio.descr_offerta p").text.strip()

        data_list.append({
            "commodity": commodity, 
            "data": data,
            "comune": comune,
            "nome_offerta": nome_offerta,
            "cod_offerta": cod_offerta,
            "venditore": venditore,
            "data_inizio": inizio,
            "data_fine": fine,
            "spesa_annua(euro)": spesa_annua,
            "materia_gas_naturale(euro)": materia_gas_naturale,
            "trasporto_gestione_contatore(euro)": trasporto_gestione_contatore,
            "oneri_sistema(euro)": oneri_sistema,
            "spesa_netto_imposte(euro)": spesa_netto_imposte,
            "imposte(euro)": imposte,
            "iva_5(euro)": iva_5,
            "iva_10(euro)": iva_10,
            "iva_22(euro)": iva_22,
            "tipo_offerta": tipo_offerta,
            "classe_contatore": classe_contatore, 
            "consumo_annuo(Smc)": consumo_annuo,
            "tipo_cliente": tipo_cliente,
            "descrizione": descrizione
        })

        driver.back()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  

        try:
            popup_close_button = driver.find_element(By.ID, "button-close-customer")
            driver.execute_script("arguments[0].scrollIntoView(true);", popup_close_button)
            popup_close_button.click()
            time.sleep(1)  
        except:
            pass  

        boxes = driver.find_elements(By.CSS_SELECTOR, "div[class*='boxRisultato_shadow']")[1:]
        index += 1  

    data_list_totale.extend(data_list)  

    driver.quit()

In [21]:
df03_GAS = pd.DataFrame(data_list_totale)
df03_GAS = df03_GAS.sort_values(by=['comune', 'spesa_annua(euro)'], ascending=[True, True])
df03_GAS.insert(1, 'profilo_cliente', '03')
df03_GAS.insert(20, 'persone_abitazione', 4)
df03_GAS.insert(21, 'superficie_abitazione(m2)', 'Da 100 a 115')
df03_GAS

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,iva_5(euro),iva_10(euro),iva_22(euro),tipo_offerta,classe_contatore,persone_abitazione,superficie_abitazione(m2),consumo_annuo(Smc),tipo_cliente,descrizione
0,Gas,03,18/10/2024,39100 - Bolzano,FAMILY GAS WEB,000139GSFML01XXW71987W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"2.553,12",...,"0,00","39,46","382,12",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta per clienti retail con tipologia di ut...
1,Gas,03,18/10/2024,39100 - Bolzano,GAS NATURALE FIX,025767GSFML01XXXXXGASDOMFCCC0924,ARGOS S.P.A.,08/10/2024,11/11/2024,"2.586,45",...,"0,00","40,57","385,94",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta a prezzo fisso.
2,Gas,03,18/10/2024,39100 - Bolzano,FAMILY GAS,000139GSFML01XXW71983W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"2.634,98",...,"0,00","40,9","394,02",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta per clienti retail con tipologia di ut...
3,Gas,03,18/10/2024,39100 - Bolzano,GAS FISSA WEB,025767GSVML01XXXXXGASDOMFWEB1024,ARGOS S.P.A.,08/10/2024,11/11/2024,"2.642,47",...,"0,00","41,77","393,67",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta a prezzo fisso.
4,Gas,03,18/10/2024,39100 - Bolzano,Segno-V-Gas SmartX,000453GSFML01XX23627SegnoXVXGSEV,SEGNOVERDE S.P.A.,17/10/2024,31/10/2024,"2.645,90",...,"0,00","41,1","395,61",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta fissa di gas naturale dedicata a clien...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Gas,03,18/10/2024,39100 - Bolzano,FORMIDABILE MOVE GAS,000142GSFML01XXResECAn0000095805,Enel Energia S.p.A.,27/09/2024,29/10/2024,"3.508,22",...,"0,00","56,26","521,02",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,E' l'offerta di Gas naturale dedicata ai siti ...
134,Gas,03,18/10/2024,39100 - Bolzano,BAROCCO FAMILY GAS FIX,001260GSFML11XX00BRCFMLYGASFIX24,Union Gas Metano S.p.A.,06/10/2024,05/11/2024,"3.512,28",...,"0,00","56,84","520,62",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta per le utenze domestiche con il prezzo...
135,Gas,03,18/10/2024,39100 - Bolzano,CASA SICURA COSTANTE,029456GSFML01XX23126CASAXSICUEPN,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"3.524,85",...,"0,00","55,78","524,98",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta fissa di gas naturale dedicata a clien...
136,Gas,03,18/10/2024,39100 - Bolzano,CASA PLUS FIX,000453GSFML01XX22799CASAXPLUSSEV,SEGNOVERDE S.P.A.,13/09/2024,31/10/2024,"3.570,77",...,"0,00","55,3","534,21",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta fissa di gas naturale dedicata a clien...


In [22]:
df_GAS = pd.concat([df01_GAS, df02_GAS, df03_GAS], ignore_index=True)
df_GAS

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,iva_5(euro),iva_10(euro),iva_22(euro),tipo_offerta,classe_contatore,persone_abitazione,superficie_abitazione(m2),consumo_annuo(Smc),tipo_cliente,descrizione
0,Gas,01,18/10/2024,39100 - Bolzano,GAS NATURALE FIX,025767GSFML01XXXXXGASDOMFCCC0924,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.301,23",...,"0,00","40,57","154,18",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta a prezzo fisso.
1,Gas,01,18/10/2024,39100 - Bolzano,FAMILY GAS WEB,000139GSFML01XXW71987W4124W00000,Dolomiti Energia SpA,07/10/2024,07/11/2024,"1.301,26",...,"0,00","39,46","156,37",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta per clienti retail con tipologia di ut...
2,Gas,01,18/10/2024,39100 - Bolzano,E.ON GasClick - Amico new,000362GSFML01XXMP04W0DA00000A000,E.ON Energia S.p.A.,11/10/2024,24/10/2024,"1.306,01",...,"0,00","42,68","150,86",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta di gas naturale per la casa con un pre...
3,Gas,01,18/10/2024,39100 - Bolzano,NeN Gas Special 36,029748GSFML01XX241010GDBSP360000,YADA ENERGIA S.R.L,10/10/2024,23/10/2024,"1.316,94",...,"0,00","43,06","158,55",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Con NeN paghi una rata fissa mensile calcolata...
4,Gas,01,18/10/2024,39100 - Bolzano,GAS FISSA WEB,025767GSVML01XXXXXGASDOMFWEB1024,ARGOS S.P.A.,08/10/2024,11/11/2024,"1.320,89",...,"0,00","41,77","155,35",Fissa,Portata < G6,1,Da 40 a 55,1.092,Domestico,Offerta a prezzo fisso.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Gas,03,18/10/2024,39100 - Bolzano,FORMIDABILE MOVE GAS,000142GSFML01XXResECAn0000095805,Enel Energia S.p.A.,27/09/2024,29/10/2024,"3.508,22",...,"0,00","56,26","521,02",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,E' l'offerta di Gas naturale dedicata ai siti ...
416,Gas,03,18/10/2024,39100 - Bolzano,BAROCCO FAMILY GAS FIX,001260GSFML11XX00BRCFMLYGASFIX24,Union Gas Metano S.p.A.,06/10/2024,05/11/2024,"3.512,28",...,"0,00","56,84","520,62",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta per le utenze domestiche con il prezzo...
417,Gas,03,18/10/2024,39100 - Bolzano,CASA SICURA COSTANTE,029456GSFML01XX23126CASAXSICUEPN,ENERGIA PULITA S.R.L.,15/10/2024,15/11/2024,"3.524,85",...,"0,00","55,78","524,98",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta fissa di gas naturale dedicata a clien...
418,Gas,03,18/10/2024,39100 - Bolzano,CASA PLUS FIX,000453GSFML01XX22799CASAXPLUSSEV,SEGNOVERDE S.P.A.,13/09/2024,31/10/2024,"3.570,77",...,"0,00","55,3","534,21",Fissa,Portata < G6,4,Da 100 a 115,2.284,Domestico,Offerta fissa di gas naturale dedicata a clien...


In [23]:
# Gestione dei separatori di migliaia e decimali e conversione in valori numerici
df_GAS['consumo_annuo(Smc)'] = pd.to_numeric(df_GAS['consumo_annuo(Smc)'].str.replace('.', ''))
 
for col in df_GAS.columns:
    if 'euro' in col:
        df_GAS[col] = df_GAS[col].str.replace('.', '')
        df_GAS[col] = pd.to_numeric(df_GAS[col].str.replace('.', '').str.replace(',', '.'))
 
# Gestione colonne data
df_GAS['data'] = pd.to_datetime(df_GAS['data'], format='%d/%m/%Y').dt.date
df_GAS['data_inizio'] = pd.to_datetime(df_GAS['data_inizio'], format='%d/%m/%Y').dt.date
df_GAS['data_fine'] = pd.to_datetime(df_GAS['data_fine'], format='%d/%m/%Y').dt.date

df_GAS = df_GAS.sort_values(by=['comune', 'profilo_cliente', 'spesa_annua(euro)'], ascending=[True, True, True])
 
df_GAS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   commodity                           420 non-null    object 
 1   profilo_cliente                     420 non-null    object 
 2   data                                420 non-null    object 
 3   comune                              420 non-null    object 
 4   nome_offerta                        420 non-null    object 
 5   cod_offerta                         420 non-null    object 
 6   venditore                           420 non-null    object 
 7   data_inizio                         420 non-null    object 
 8   data_fine                           420 non-null    object 
 9   spesa_annua(euro)                   420 non-null    float64
 10  materia_gas_naturale(euro)          420 non-null    float64
 11  trasporto_gestione_contatore(euro)  420 non-n

In [24]:
df_GAS

,commodity,profilo_cliente,data,comune,nome_offerta,cod_offerta,venditore,data_inizio,data_fine,spesa_annua(euro),...,iva_5(euro),iva_10(euro),iva_22(euro),tipo_offerta,classe_contatore,persone_abitazione,superficie_abitazione(m2),consumo_annuo(Smc),tipo_cliente,descrizione
0,Gas,01,2024-10-18,39100 - Bolzano,GAS NATURALE FIX,025767GSFML01XXXXXGASDOMFCCC0924,ARGOS S.P.A.,2024-10-08,2024-11-11,1301.23,...,0.0,40.57,154.18,Fissa,Portata < G6,1,Da 40 a 55,1092,Domestico,Offerta a prezzo fisso.
1,Gas,01,2024-10-18,39100 - Bolzano,FAMILY GAS WEB,000139GSFML01XXW71987W4124W00000,Dolomiti Energia SpA,2024-10-07,2024-11-07,1301.26,...,0.0,39.46,156.37,Fissa,Portata < G6,1,Da 40 a 55,1092,Domestico,Offerta per clienti retail con tipologia di ut...
2,Gas,01,2024-10-18,39100 - Bolzano,E.ON GasClick - Amico new,000362GSFML01XXMP04W0DA00000A000,E.ON Energia S.p.A.,2024-10-11,2024-10-24,1306.01,...,0.0,42.68,150.86,Fissa,Portata < G6,1,Da 40 a 55,1092,Domestico,Offerta di gas naturale per la casa con un pre...
3,Gas,01,2024-10-18,39100 - Bolzano,NeN Gas Special 36,029748GSFML01XX241010GDBSP360000,YADA ENERGIA S.R.L,2024-10-10,2024-10-23,1316.94,...,0.0,43.06,158.55,Fissa,Portata < G6,1,Da 40 a 55,1092,Domestico,Con NeN paghi una rata fissa mensile calcolata...
4,Gas,01,2024-10-18,39100 - Bolzano,GAS FISSA WEB,025767GSVML01XXXXXGASDOMFWEB1024,ARGOS S.P.A.,2024-10-08,2024-11-11,1320.89,...,0.0,41.77,155.35,Fissa,Portata < G6,1,Da 40 a 55,1092,Domestico,Offerta a prezzo fisso.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Gas,03,2024-10-18,39100 - Bolzano,FORMIDABILE MOVE GAS,000142GSFML01XXResECAn0000095805,Enel Energia S.p.A.,2024-09-27,2024-10-29,3508.22,...,0.0,56.26,521.02,Fissa,Portata < G6,4,Da 100 a 115,2284,Domestico,E' l'offerta di Gas naturale dedicata ai siti ...
416,Gas,03,2024-10-18,39100 - Bolzano,BAROCCO FAMILY GAS FIX,001260GSFML11XX00BRCFMLYGASFIX24,Union Gas Metano S.p.A.,2024-10-06,2024-11-05,3512.28,...,0.0,56.84,520.62,Fissa,Portata < G6,4,Da 100 a 115,2284,Domestico,Offerta per le utenze domestiche con il prezzo...
417,Gas,03,2024-10-18,39100 - Bolzano,CASA SICURA COSTANTE,029456GSFML01XX23126CASAXSICUEPN,ENERGIA PULITA S.R.L.,2024-10-15,2024-11-15,3524.85,...,0.0,55.78,524.98,Fissa,Portata < G6,4,Da 100 a 115,2284,Domestico,Offerta fissa di gas naturale dedicata a clien...
418,Gas,03,2024-10-18,39100 - Bolzano,CASA PLUS FIX,000453GSFML01XX22799CASAXPLUSSEV,SEGNOVERDE S.P.A.,2024-09-13,2024-10-31,3570.77,...,0.0,55.30,534.21,Fissa,Portata < G6,4,Da 100 a 115,2284,Domestico,Offerta fissa di gas naturale dedicata a clien...


In [25]:
if df_GAS['comune'].nunique() == 1:
    df_GAS.to_excel(f'ARERA_{df_GAS["commodity"].iloc[0].title().replace(" ", "")}-{df_GAS["comune"].iloc[0].title().replace(" ", "")}-{df_GAS["data"].iloc[0].strftime("%Y-%m-%d")}.xlsx', index=False)
else:
    df_GAS.to_excel(f'ARERA_{df_GAS["commodity"].iloc[0].title().replace(" ", "")}-{df_GAS["data"].iloc[0].strftime("%Y-%m-%d")}.xlsx', index=False)